[View in Colaboratory](https://colab.research.google.com/github/tariqdaouda/Mariana/blob/V2-dev/Mariana_basics_example.ipynb)

**This is a quick Mariana example to showcase:**


* The installation
* How to a neural network with a skip connection
* How to visualize the architecture
* How to train and test a model
* How to get any layer's output
* How create mix-in functions for debugging and  train several outputs
* Get the gradients and outputs of any layer for debugging and interpretation




In [2]:
#!kill -9 -1
!pip install git+https://github.com/Theano/Theano
!pip install git+https://github.com/Lasagne/Lasagne
!pip install git+https://github.com/tariqdaouda/Mariana

  Cloning https://github.com/Theano/Theano to /tmp/pip-req-build-LImt5D
  Running setup.py bdist_wheel for Theano ... - \ | / - \ | / - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-55Rdzf/wheels/64/f2/f4/6b1f50baf18aca2eab5d9b5a431b90e3d8be4711c8f7457eb7
Successfully built Theano
  Cloning https://github.com/Lasagne/Lasagne to /tmp/pip-req-build-pocfxC
  Running setup.py bdist_wheel for Lasagne ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-2482Ax/wheels/c4/20/90/9f7242c381402829c5918261e3eb51a87bc1d8521456749b57
Successfully built Lasagne
  Cloning https://github.com/tariqdaouda/Mariana to /tmp/pip-req-build-Bm1ZWE
    100% |████████████████████████████████| 7.1MB 3.0MB/s 
    100% |████████████████████████████████| 81kB 20.3MB/s 
  Running setup.py bdist_wheel for Mariana ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-nnk3G1/wheels/1f/ca/1b/22d3ed05d19c6b64cb8bd1e926058a1cf23ede03cc3a0d2eda
  Running setup.py bdist_wheel for

- done
  Stored in directory: /content/.cache/pip/wheels/ad/ab/a0/860907dc9ef89c041fbb4627dc11d9c05de0808939b6539b3e
Successfully built Mariana pyGeno simplejson rabaDB


In [14]:
import Mariana.layers as ML
import Mariana.scenari as MS
import Mariana.costs as MC
import Mariana.activations as MA
import Mariana.regularizations as MR

import Mariana.settings as MSET
import numpy

ls = MS.GradientDescent(lr = 0.01, momentum=0.9)
cost = MC.NegativeLogLikelihood()

inp = ML.Input(28*28, name = "InputLayer")
h1 = ML.Hidden(300, activation = MA.ReLU(), name = "Hidden1", regularizations = [ MR.L1(0.0001) ])
h2 = ML.Hidden(300, activation = MA.ReLU(), name = "Hidden2", regularizations = [ MR.L1(0.0001) ])
o = ML.SoftmaxClassifier(10, learningScenari = [ls], cost = cost, name = "Probabilities1")
o.addNote("This is a note on the output layer", "I am the output layer")

#Connecting layers
inp > h1 > h2
concat = ML.C([inp, h2], name="Concatenation")

MLP_skip = concat > o
MLP_skip.addNote("This is a note on the model", "This is the skip connection example. Notes make it easier to collaborate")
MLP_skip.init()


>|\/| /-\ |-> | /-\ |\| /-\>


In [13]:
#Visualizing
MLP_skip.view("mySkipMLP")


In [0]:
print "->train:\n", MLP_skip["Probabilities"].train({"InputLayer.inputs": [numpy.ones(784)], "Probabilities.targets": [1]})
print "---"
print "->test:\n", MLP_skip["Probabilities"].test({"InputLayer.inputs": [numpy.ones(784)], "Probabilities.targets": [1]})
print "---"

print "->the output of the first hidden layer, on the stream 'train' (has regularisations):\n", MLP_skip["Hidden1"].propagate["train"]({"InputLayer.inputs": [numpy.ones(784)]})
print "---"
print "->the output of the last layer, on the stream 'test' (no regularisations):\n", MLP_skip["Probabilities"].propagate["test"]({"InputLayer.inputs": [numpy.ones(784)]})
print "---"

#mix-in function: output on the second hidden + test score
f =  MLP_skip["Hidden2"].propagate["test"] + MLP_skip["Probabilities"].test
print "->mix-in outputs:\n", f({"InputLayer.inputs": [numpy.ones(784)], "Probabilities.targets": [1]})

In [15]:
ls = MS.GradientDescent(lr = 0.01, momentum=0.9)
cost = MC.NegativeLogLikelihood()

inp = ML.Input(28*28, name = "InputLayer")
h1 = ML.Hidden(300, activation = MA.ReLU(), name = "Hidden1", regularizations = [ MR.L1(0.0001) ])
h2 = ML.Hidden(300, activation = MA.ReLU(), name = "Hidden2", regularizations = [ MR.L1(0.0001) ])
o1 = ML.SoftmaxClassifier(10, learningScenari = [ls], cost = cost, name = "Probabilities1")
o2 = ML.SoftmaxClassifier(3, learningScenari = [ls], cost = cost, name = "Probabilities2")

#Connecting layers
inp > h1 > h2
concat = ML.C([inp, h2], name="Concatenation")

h1 > o1
MLP_skip = concat > o2

MLP_skip.addNote("This is a note on the model", "This model has two outputs")
MLP_skip.init()


>|\/| /-\ |-> | /-\ |\| /-\>


In [17]:
#Visualizing
MLP_skip.view("mySkipMLP")

In [28]:
print "->train on fisrt output:\n", MLP_skip["Probabilities1"].train({"InputLayer.inputs": [numpy.ones(784)], "Probabilities1.targets": [1]})
print "---"

#mix-in function: train on both outputs
f =  MLP_skip["Probabilities1"].train + MLP_skip["Probabilities2"].train
print "->mix-in outputs:\n", f({"InputLayer.inputs": [numpy.ones(784)], "Probabilities1.targets": [1], "Probabilities2.targets": [0]})

->train on fisrt output:
OrderedDict([('Probabilities1.drive.train', array(9.08210724e-06))])
---
->mix-in outputs:
OrderedDict([('Probabilities1.drive.train', array(2.71914144e-06)), ('Probabilities2.drive.train', array(3.11565855))])


In [30]:
#Get the gradients for debugging and interpretation
MLP_skip["Probabilities1"].train.getGradients({"InputLayer.inputs": [numpy.ones(784)], "Probabilities1.targets": [1]})

OrderedDict([('Hidden1.b',
              array([ 0.00000000e+00,  0.00000000e+00, -1.51440640e-07,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00, -2.11944325e-07,  0.00000000e+00,
                     -8.45051287e-08, -3.29868775e-07,  0.00000000e+00,  0.00000000e+00,
                     -1.93888435e-08,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                      0.00000000e+00, -2.65512340e-07,  8.64366131e-08,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.24948273e-08,
                     -2.46019679e-07,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                      0.00000000e+00, -1.22970078e-08,  0.00000000e+00,  1.97720300e-07,
                      0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00, -1.65620081e-07,  0.00000000

In [29]:
#Get the updates for debugging and interpretation
MLP_skip["Probabilities1"].train.getUpdates({"InputLayer.inputs": [numpy.ones(784)], "Probabilities1.targets": [1]})

OrderedDict([('Hidden1.b',
              array([ 0.00000000e+00,  0.00000000e+00,  1.34647880e-03,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00,  8.48849536e-04,  0.00000000e+00,
                     -4.02560261e-05,  2.22665159e-03,  0.00000000e+00,  0.00000000e+00,
                     -1.96807503e-04,  0.00000000e+00, -1.69743639e-03,  0.00000000e+00,
                      0.00000000e+00,  2.17963700e-03, -1.07524098e-03,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00, -2.59541859e-03, -1.61847860e-03,
                      2.26993545e-03, -1.04499160e-03, -1.73618232e-03,  0.00000000e+00,
                      0.00000000e+00, -2.18082692e-04,  0.00000000e+00, -1.88735617e-03,
                      0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                      0.00000000e+00,  0.00000000e+00,  9.94986334e-04,  0.00000000